<a href="https://colab.research.google.com/github/sahoosudipto/MiniProj_Distributed_Mat_Mul/blob/main/MPI_Based_Distributed_Matrix_Multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 12.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.1-cp310-cp310-linux_x86_64.whl size=4266351 sha256=c70c1c40be40760bce2ba45032f743d9b054522526f5082c8e3bed270ef6f3d7
  Stored in directory: /root/.cache/pip/wheels/3c/ca/13/13218a83854023ccec184e3af482f0f038b434aa32c19afee8
Successfully built mpi4py


In [ ]:
import numpy as np
import time
from mpi4py import MPI

def serial_matrix_multiply(A, B):
    """
    Standard serial matrix multiplication implementation

    Args:
        A (np.ndarray): First input matrix
        B (np.ndarray): Second input matrix

    Returns:
        np.ndarray: Resulting matrix after multiplication
    """
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape

    if cols_A != rows_B:
        raise ValueError("Matrix dimensions incompatible for multiplication")

    result = np.zeros((rows_A, cols_B))


    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]

    return result

def distribute_matrix(matrix, comm, root=0):

    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == root:
        matrix_chunks = np.array_split(matrix, size)
    else:
        matrix_chunks = None

    local_chunk = comm.scatter(matrix_chunks, root=root)
    return local_chunk

def parallel_matrix_multiply(A, B, comm):
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Distribute matrices
    local_A = distribute_matrix(A, comm)
    local_B = distribute_matrix(B, comm)

    # multiplication
    local_result = np.dot(local_A, local_B)

    # Gather results
    global_result = comm.gather(local_result, root=0)

    return np.concatenate(global_result) if rank == 0 else None

In [ ]:
#test regular matrix multiplication
start_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
A = np.random.rand(200, 200)
B = np.random.rand(200, 200)
result = serial_matrix_multiply(A, B)
end_time = time.time()
execution_time = end_time - start_time
print("Distributed multiplication completed")
print(f"Execution time: {execution_time} seconds")


Distributed multiplication completed
Execution time: 11.247038125991821 seconds


In [ ]:
#test parallel matrix multiplication
start_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
A = np.random.rand(200, 200)
B = np.random.rand(200, 200)
result = parallel_matrix_multiply(A, B, comm)
end_time = time.time()
execution_time = end_time - start_time
if rank == 0:
    print("Distributed multiplication completed")
    print(f"Execution time: {execution_time} seconds")


Distributed multiplication completed
Execution time: 0.009556055068969727 seconds


In [ ]:
#test benchmark matrix multiplication
start_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
A = np.random.rand(1000, 1000)
B = np.random.rand(1000, 1000)
result = parallel_matrix_multiply(A, B, comm)
end_time = time.time()
execution_time = end_time - start_time
if rank == 0:
  print("Distributed multiplication completed")
  print(f"Execution time: {execution_time} seconds")



#File structured

## distributed matrix multiplication

In [ ]:
%%writefile dist_matrix_mult.py
from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Function to perform serial matrix multiplication
def serial_matrix_mult(A, B):
    return np.dot(A, B)

# Function to perform distributed matrix multiplication
# Function to perform distributed matrix multiplication
def distributed_matrix_mult(A, B, m, n, p):
    # Determine local rows for each process
    local_rows = m // size
    start_row = rank * local_rows
    end_row = start_row + local_rows

    # Scatter matrix A
    local_A = np.empty((local_rows, n))
    comm.Scatter(A, local_A, root=0)

    # Broadcast matrix B
    B = comm.bcast(B, root=0)

    # Perform local matrix multiplication
    local_C = np.dot(local_A, B)

    # Gather results
    C = np.empty((m, p))
    comm.Gather(local_C, C, root=0)

    return C


if __name__ == "__main__":
    # ... (same as before) ...

## test file

In [ ]:
%%writefile test_matrix_mult.py
import unittest
import numpy as np
from dist_matrix_mult import serial_matrix_mult, distributed_matrix_mult
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

class TestMatrixMult(unittest.TestCase):

    def test_serial_mult(self):
        A = np.array([[1, 2], [3, 4]])
        B = np.array([[5, 6], [7, 8]])
        C_expected = np.array([[19, 22], [43, 50]])
        C_actual = serial_matrix_mult(A, B)
        np.testing.assert_array_equal(C_actual, C_expected)

    def test_distributed_mult(self):
        m = 4
        n = 4
        p = 4
        A = np.random.rand(m, n)
        B = np.random.rand(n, p)

        if rank == 0:
            C_serial = serial_matrix_mult(A, B)

        C_dist = distributed_matrix_mult(A, B, m, n, p)

        if rank == 0:
            np.testing.assert_allclose(C_serial, C_dist, rtol=1e-5, atol=1e-5)

if __name__ == '__main__':
    unittest.main()